In [1]:
import os
os.chdir(os.getcwd()[:os.getcwd().rfind('\\')])

In [2]:
import numpy as np
from src.utils.preprocessing_utils import load_image, preprocess_image, get_bounding_boxes, get_corners, get_areas, filter_by_area, get_boxes_sides_length, filter_by_sides
from src.utils.frontend_utils import input_for_frontend
from src.utils.data_utils import zero_padding, remove_extra_space_around_characters, get_characters
import cv2
from skimage import io
import os
from tensorflow import keras
import numpy as np
from skimage.transform import resize
from skimage import io, color
from sklearn.preprocessing import LabelEncoder
import statistics
from src.utils.preprocessing_utils import filter_boxes_by_models_predictions, get_all_leters, project_y_letters, find_distances_between_coordinates, get_lines, get_sorted_lines, get_text_lines
from src.utils.frontend_utils import get_model_response

## Load Model

In [3]:
GEO_ALPHABET = {'ა': 1, 'ბ': 2, 'გ': 3, 'დ': 4, 'ე': 5, 'ვ': 6, 
                'ზ': 7, 'თ': 8, 'ი': 9, 'კ': 10, 'ლ': 11, 'მ': 12, 
                'ნ': 13, 'ო': 14, 'პ': 15, 'ჟ': 16, 'რ': 17, 'ს': 18, 
                'ტ': 19, 'უ': 20, 'ფ': 21, 'ქ': 22, 'ღ': 23, 'ყ': 24, 
                'შ': 25, 'ჩ': 26, 'ც': 27, 'ძ': 28, 'წ': 29, 'ჭ': 30, 
                'ხ': 31, 'ჯ': 32, 'ჰ': 33} 

model_name = 'models/first_cnn/first_cnn.h5'
label_encoder_name = 'models/first_cnn/first_cnn_label_encoder.npy'

model = keras.models.load_model(model_name)
le = LabelEncoder()
le.classes_ = np.load(label_encoder_name, allow_pickle=True)
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))

## Preprocess Image

In [4]:
image_file = 'data/raw/0002.jpg'

img = load_image(image_file)
binary_image = preprocess_image(img)
# characteristics
bounding_boxes = get_bounding_boxes(binary_image)
corners = get_corners(bounding_boxes)
areas = get_areas(corners)
widths, heights = get_boxes_sides_length(corners)

# filters
filtered_areas, filtered_corners, area_mask = filter_by_area(areas, corners, filter_value = 0)
filtered_widths, filtered_heights = np.array(widths)[np.array(area_mask)].tolist(), np.array(heights)[np.array(area_mask)].tolist()

filtered_corners, side_mask = filter_by_sides(filtered_corners, filtered_widths, filtered_heights)
filtered_widths, filtered_heights = np.array(filtered_widths)[np.array(side_mask)].tolist(), np.array(filtered_heights)[np.array(side_mask)].tolist()
filtered_areas = np.array(filtered_areas)[np.array(side_mask)].tolist()

filtered_areas, filtered_corners, area_mask = filter_by_area(filtered_areas, filtered_corners, filter_value = np.mean(filtered_areas)/2)
filtered_widths, filtered_heights = np.array(filtered_widths)[np.array(area_mask)].tolist(), np.array(filtered_heights)[np.array(area_mask)].tolist()

In [5]:
# %%timeit
characters = get_characters(binary_image, filtered_corners)

## Predictions

In [6]:
predictions = []
for c in characters:
    preprocessed = zero_padding(remove_extra_space_around_characters(c, extra_space_value=0))
    image_cnn = np.zeros(shape=(1, 28, 28))
    image_cnn[0] = zero_padding(remove_extra_space_around_characters(c, extra_space_value=0))
    prediction = model.predict(image_cnn, verbose=False)
    letter, confidence = np.argmax(prediction), np.max(prediction)
    
    predictions.append((le_name_mapping[letter], confidence))

In [7]:
model_response = get_model_response(predictions, filtered_corners)

### Get only confident boxes

In [8]:
f_corners, f_predictions, prediction_mask = filter_boxes_by_models_predictions(filtered_corners, predictions, 0.9)

In [9]:

all_letters = get_all_leters(f_corners)
y_letters = project_y_letters(all_letters)
coord_distances = find_distances_between_coordinates(y_letters)
lines = get_lines(coord_distances, all_letters, y_letters)
sorted_lines = get_sorted_lines(lines)
text_lines = get_text_lines(sorted_lines, model_response)

In [10]:
sorted_text_lines = [sorted(tl, key=lambda x: x['corners'][0][0]) for tl in text_lines]

joined_text_lines = ["".join([s['letter'] for s in stl]) for stl in sorted_text_lines ]

mean_spaces_between_chars = [statistics.mean([abs(stl[i]['corners'][2][0] - stl[i+1]['corners'][0][0]) for i in range(len(stl)-1)]) for stl in sorted_text_lines]

result_text = ""
for i in range(len(sorted_text_lines)):
    sentence = sorted_text_lines[i]
    mean_space = mean_spaces_between_chars[i]
    sent = ""
    for i in range(len(sentence)-1):
        sent += sentence[i]['letter']
        if abs(sentence[i]['corners'][2][0] - sentence[i+1]['corners'][0][0]) > mean_space:
            sent+= " "
    sent += sentence[-1]['letter']
    result_text += sent + '\n'

print(result_text)

ჭ ნ ნ შენავტლი
ნ ქძ ჯყ კძცაუქუნის კ ნანუვარი უაღრუსად წ ყოფიუძ რი პურიოდია
ქართლ ს ისტორიაში უს არის დიდ დ გ ნ თლუბული ს ხელმწიფრ
მრღვაწის ქართველი მუფის ვანტანგ ყკ ის მშართვულობის და მუ
ფობლც ხანა რომულ ც შვუყნიც დ მოუქიდებლოჭ ბიცა და პოლიტიქუ
რი ერთიანობიც თვის ბრძოლითა და დიდი ქულტურლლ სუ აგანმგწათ
ლებლ ნ რ მვრებით ხასიუდება მაჰმადიანთა ქალმომრუობისბცირო
ბუბში ვახწაწგი უროვნულ ქრიცტიანული პოლიწიქიც მუდროშუდ გ
მოვიდა მ ნ ქარგად უწყოდა რომ უროვნფლი სახის შენარჩუნება
ფიზიქლრი არსუბობისათვის ბრძოლის გარდა ქვუყნის იცტორიუჟ ლ
ქლლტურულ ჟი მუმქვიდრეობიც მოვლ ცა და მიც შემდგომ გამდიდრუ
ბ ც ც მოითხოვდა ცწორუდ ამ მჯრივაა საყურ დღებო ვახწანგის
ოღგორც ქულტურული მრღვ წის და განმანათლებლის მუცნიურიცა
დ მთარგმნულიც მოღვაწუობ
ვ ჯწანგი რის უშუ ლო წარმმართვული და აშტილრი მონაწილე
იმ საერთო ურრვნულ ქლლტურული პროცუსიც რომულცაც შართულ
ანმანულუ ბლშბჯს უწრდე გ ე ნ ცწორუდ გაჯოტჯი ხგწა იყო უც ჯნა
სუთი დაუღალავი მედგარი მლშაობა რომელსაც ურთი საზოგ დო
მიმ რთულუბა ჰქონდ დ ფართო ღრმ ნიად გი უპყრ უფლუბაც გვა
შლევ